In [ ]:
from pandas import DataFrame
from pandas import read_csv
from bigml.api import BigML

In [ ]:
df = read_csv('cs-training.csv', header=0, index_col=0)
df.fillna(0, inplace=True)
df['NumberOfTimeTotalDelay'] = df['NumberOfTime30-59DaysPastDueNotWorse']
                                + df['NumberOfTimes90DaysLate']
                                + df['NumberOfTime60-89DaysPastDueNotWorse']
df['isOld'] =df['age']>70
df.to_csv('train_full.csv')

In [ ]:
df = read_csv('cs-test.csv', header=0, index_col=0)
df.fillna(0, inplace=True)
df['NumberOfTimeTotalDelay'] = df['NumberOfTime30-59DaysPastDueNotWorse']
                                + df['NumberOfTimes90DaysLate']
                                + df['NumberOfTime60-89DaysPastDueNotWorse']
df['isOld'] =df['age']>70
df.to_csv('test_full.csv')

In [ ]:
api = BigML(project='project/5d94a344eba31d460c000235')

In [ ]:
source_train = api.create_source('train_full.csv')
origin_dataset = api.create_dataset(source_train)
source_test = api.create_source('test_full.csv')
test_dataset = api.create_dataset(source_test)

In [ ]:
train_dataset = api.create_dataset(
    origin_dataset, {"name": "Training_split_Jupyter",
                     "sample_rate": 0.8, "seed": "my seed"})
val_dataset = api.create_dataset(
    origin_dataset, {"name": "Validation_split_Jupyter",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [ ]:
ensemble = api.create_ensemble(train_dataset,{"name": "ensemble_Jupyter",
     "objective_field": "SeriousDlqin2yrs"})

In [ ]:
evaluation = api.create_evaluation(ensemble, val_dataset)

In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, test_dataset, {
    "name": "ensemble_prediction_Jupyter", "all_fields": True,
    "header": True,
    "confidence": True,
    "probabilities": True})
api.ok(batch_prediction)

In [ ]:
api.download_batch_prediction(batch_prediction,
    filename='ensemble_prediction_Jupyter.csv')

In [ ]:
df = read_csv('ensemble_prediction_Jupyter.csv', header=0, index_col=0)
df2 =df [['1 probability']]
df2.index.names = ['Id']
df2.rename(columns={"1 probability": "Probability"},inplace=True)

In [ ]:
df2.to_csv('prediction_kaggle.csv')